# Lab 2.2 - Condensed Nearest Neighbors

Celem zadania jest obserwacja wpływu kompresji Condensed Nearest Neighbours, w skrócie CNN, (najlepiej opisana w oryginalnej, bardzo krótkiej publikacji - patrz załącznik) na pracę klasyfikatora k-NN. Alternatywny (ale moim zdaniem miejscami niepełny) opis znajduje się także na Wikipedii (https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm#CNN_for_data_reduction) - tam też można znaleźć lepiej wyjaśniające problem obrazki.

**Wykonanie rozwiązań: Marcin Przewięźlikowski**

https://github.com/mprzewie/ml_basics_course

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from imblearn.under_sampling import CondensedNearestNeighbour
from typing import Dict, Tuple
import cv2
from dataset_ops import (
    load_dataset, visualize_dataset, slice_dataset, sliced_dataset_name
)
from copy import deepcopy
from collections import OrderedDict

## Przygotowanie datasetów

Korzystając np. z omawianej na zajęciach metody "z paintem" (lub innej), przygotuj 3 różne (ciekawe) dwuwymiarowe zbiory danych. Przynajmniej raz powinna wystąpić każda z poniższych sytuacji:
- 3 lub więcej różnych klas
- klasy dobrze odseparowane w danym regionie;
- klasy częściowo przemieszane, nachodzące na siebie;
- wyspa jednej klasy wewnątrz regionu drugiej;m
- różne gęstości punktów wewnątrz poszczególnych klas;
- nieregularne kształty obszaru jednej z klas;
- niesymetryczny kształt całego zbioru (np. podłużne wrzeciono).

Jeżeli korzystasz z metody "z paintem" to pamiętaj, by przed zapisaniem zbioru nałożyć na punkty niewielki szum. Przygotowane zbiory możesz wykorzystać w obu zadaniach domowych.


In [ ]:
orig_datasets = {}
n_datasets = 3
x1_size, x2_size = (100, 100)
for i in range(3):
    X, y = load_dataset(f"ds_{i + 1}.png", space_size=(x1_size, x2_size), dropout=0.8)
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    orig_datasets[f"{i}_train"] = X_train, y_train
    orig_datasets[f"{i}_test"] = X_test, y_test

In [ ]:
for name, (X, y) in orig_datasets.items():
    print(name)
    visualize_dataset(X, y)
    plt.show()

Podobnie jak w poprzednim zadaniu z pary (Metric Learning) przygotowujemy zbiory danych, a następnie obserwujemy wygląd granicy decyzyjnej i % skuteczność klasyfikacji. Tym razem jednak korzystamy z następujących klasyfikatorów:

In [ ]:
all_coords = np.arange(0, 100)
X_all = np.array([[[x0, x1] for x1 in all_coords] for x0 in all_coords]).reshape(-1, 2)
y_all_ph = np.array([0 for _ in X_all])
all_sliced = {}
datasets = {}
for n_slices in [1]:
    for ds_name, (X, y) in orig_datasets.items():
        sliced = slice_dataset(X, y, n_slices=n_slices)
        for (x0, x1), (X_s, y_s) in sliced.items():
            datasets[sliced_dataset_name(ds_name, n_slices, x0, x1)] = X_s, y_s
    sliced_all_by_x0_x1 = slice_dataset(X_all, y_all_ph, n_slices=n_slices)
    for (x0, x1), (X_s, y_s) in sliced_all_by_x0_x1.items():
        all_sliced[(n_slices, x0, x1)] = X_s, y_s

In [ ]:
def train_knns(
    knn_prototype: KNeighborsClassifier, 
    sampling=None
) -> Dict[Tuple[int, int, int], KNeighborsClassifier]:
    knn_dict = {}
    for i in range(n_datasets):
        ds_name = sliced_dataset_name(f"{i}_train", n_slices, 0,0)
        X_train, y_train = datasets[ds_name]
        if sampling:
            X_train, y_train = sampling.fit_resample(X_train, y_train)
        if X_train.shape[0] > 0:
            knn = deepcopy(knn_prototype)
            knn.fit(X_train, y_train)
            knn_dict[(i, x0, x1)] = knn
        else:
            knn_dict[(i, x0, x1)] = None
        plt.show()
    return knn_dict

In [ ]:
knns = OrderedDict()

### zwykły k-NN z k=1 i metryką Euklidesa

In [ ]:
KNN1_NAME = "k1nn"
KNN1_PROTOTYPE = KNeighborsClassifier(n_neighbors=1, algorithm="brute")
knns[KNN1_NAME] = train_knns(KNN1_PROTOTYPE)

### CNN z k=1 i metryką Euklidesa (losowo wybierający próbki w procedurze kondensacji);

In [ ]:
KNN2_NAME = "c1nn"
KNN2_PROTOTYPE = KNeighborsClassifier(n_neighbors=1, algorithm="brute")
knns[KNN2_NAME] = train_knns(
    KNN2_PROTOTYPE,
    sampling=CondensedNearestNeighbour(n_neighbors=1, sampling_strategy="all")
)

### zwykły k-NN z k=3 i metryką Euklidesa;

In [ ]:
KNN3_NAME = "k3nn"
KNN3_PROTOTYPE = KNeighborsClassifier(n_neighbors=3, algorithm="brute")
knns[KNN3_NAME] = train_knns(KNN3_PROTOTYPE)

### CNN z k=3 i metryką Euklidesa (może być potrzebna niewielka adaptacja metody by pracowała z k>1 - zastanowić się jaka!).

In [ ]:
KNN4_NAME = "c3nn"
KNN4_PROTOTYPE = KNeighborsClassifier(n_neighbors=3, algorithm="brute")
knns[KNN4_NAME] = train_knns(
    KNN4_PROTOTYPE,
    sampling=CondensedNearestNeighbour(n_neighbors=3, sampling_strategy="all")
)

In [ ]:
for i in range(n_datasets):
    _, y_train = orig_datasets[f"{i}_train"]
    n_classes = len(np.unique(y_train))
    print("dataset", i, "n_classes =", n_classes)
    for name, classifiers in knns.items():
        print("\t",name)
        accuracies = []
        weights = []
        n_slices = 1
        knn = classifiers[(i, 0,0)]
        ds_name = sliced_dataset_name(f"{i}_test", n_slices, x0, x1)
        X_test, y_test = datasets[ds_name]
        if knn is not None and X_test.shape[0] > 1:
            weight = X_test.shape[0]
            accuracy = knn.score(X_test, y_test)
            X_all_s, _ = all_sliced[(n_slices, x0, x1)]
            y_all = knn.predict(X_all_s)
            visualize_dataset(X_all_s, y_all, n_classes)
        else: 
            weight, accuracy = 0, 0
        accuracies.append(accuracy)
        weights.append(accuracy)
        accuracies = np.array(accuracies)
        weighs = np.array(weights)
        total_acc = np.average(accuracies, weights=weights)
        print("accuracy on", i, ":", total_acc)
        plt.show()
    print()

* Algorytmy CNN radzą sobie wyraźnie gorzej niż KNN, gdyż efektywnie są KNN'ami, które biorą do treningu mniej danych. Odbija się to na dokładności predykcji, a także jest widoczne w kształcie granic między klasami. 

* Dla podanych datasetów, C1NN radzi sobie lepiej niż C3NN